In [16]:
"""
DISCLAIMER: 

This software is provided solely for educational and research purposes. 
It is not intended to provide investment advice, and no investment recommendations are made herein. 
The developers are not financial advisors and accept no responsibility for any financial decisions or losses resulting from the use of this software.
Always consult a professional financial advisor before making any investment decisions.
"""

import FreeSimpleGUI as sg
import yfinance as yf
import numpy as np
import time
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
import threading

def filter_option_dates(dates, ref_date):
    """
    Verilen tarih listesinden, ref_date'den sonraki tüm gelecek opsiyon tarihlerini alır.
    Eğer bunlardan ref_date'den itibaren 60 gün içinde olan varsa bunları,
    yoksa en yakın gelecek tarihi döndürür.
    """
    all_future = sorted([datetime.strptime(d, "%Y-%m-%d").date() 
                         for d in dates if datetime.strptime(d, "%Y-%m-%d").date() >= ref_date])
    if not all_future:
        raise ValueError("Gelecekteki opsiyon tarihi bulunamadı.")
    within_60 = [d for d in all_future if d <= ref_date + timedelta(days=60)]
    if within_60:
        if within_60[0] == ref_date and len(within_60) > 1:
            return [d.strftime("%Y-%m-%d") for d in within_60[1:]]
        return [d.strftime("%Y-%m-%d") for d in within_60]
    else:
        return [all_future[0].strftime("%Y-%m-%d")]

def yang_zhang(price_data, window=30, trading_periods=252):
    # Yang-Zhang volatilite hesaplaması
    log_ho = (price_data['High'] / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low'] / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)
    
    log_oc = (price_data['Open'] / price_data['Close'].shift(1)).apply(np.log)
    log_oc_sq = log_oc**2
    
    log_cc = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)
    log_cc_sq = log_cc**2
    
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    
    close_vol = log_cc_sq.rolling(window=window, center=False).sum() / (window - 1)
    open_vol = log_oc_sq.rolling(window=window, center=False).sum() / (window - 1)
    window_rs = rs.rolling(window=window, center=False).sum() / (window - 1)
    
    k = 0.34 / (1.34 + ((window + 1) / (window - 1)))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * np.sqrt(trading_periods)
    return result.iloc[-1]

def build_term_structure(days, ivs):
    days = np.array(days)
    ivs = np.array(ivs)
    sort_idx = days.argsort()
    days = days[sort_idx]
    ivs = ivs[sort_idx]
    spline = interp1d(days, ivs, kind='linear', fill_value="extrapolate")
    def term_spline(dte):
        if dte < days[0]:
            return ivs[0]
        elif dte > days[-1]:
            return ivs[-1]
        else:
            return float(spline(dte))
    return term_spline

def get_current_price(stock):
    try:
        todays_data = stock.history(period='1d')
        return todays_data['Close'][0]
    except Exception:
        return None

def compute_recommendation(ticker, thresholds, ref_date):
    """
    Verilen ticker için, ref_date (bugünün tarihi) üzerinden opsiyon verilerini kullanarak;
    avg_volume, iv30_rv30, ts_slope ve open interest kriterlerini kontrol eder.
    thresholds: Her biri "enabled" (bool) ve "value" (float) içeren bir sözlük.
    """
    try:
        ticker = ticker.strip().upper()
        if not ticker:
            return {"error": "Hisse sembolü girilmedi."}
        stock = yf.Ticker(ticker)
        if not stock.options:
            return {"error": f"{ticker} için opsiyon verisi bulunamadı."}
        try:
            exp_dates = filter_option_dates(stock.options, ref_date)
        except Exception as e:
            fallback_date = datetime.today().date()
            exp_dates = filter_option_dates(stock.options, fallback_date)
            ref_date = fallback_date
        options_chains = {}
        for exp_date in exp_dates:
            options_chains[exp_date] = stock.option_chain(exp_date)
        underlying_price = get_current_price(stock)
        if underlying_price is None:
            return {"error": "Hisse fiyatı alınamadı."}
        atm_iv = {}
        open_interest_check = None
        straddle = None
        i = 0
        for exp_date in exp_dates:
            chain = options_chains[exp_date]
            calls = chain.calls
            puts = chain.puts
            if calls.empty or puts.empty:
                continue
            call_diffs = (calls['strike'] - underlying_price).abs()
            call_idx = call_diffs.idxmin()
            call_iv = calls.loc[call_idx, 'impliedVolatility']
            call_oi = calls.loc[call_idx, 'openInterest']
            
            put_diffs = (puts['strike'] - underlying_price).abs()
            put_idx = put_diffs.idxmin()
            put_iv = puts.loc[put_idx, 'impliedVolatility']
            put_oi = puts.loc[put_idx, 'openInterest']
            
            atm_iv_value = (call_iv + put_iv) / 2.0
            atm_iv[exp_date] = atm_iv_value
            
            if i == 0:
                call_bid = calls.loc[call_idx, 'bid']
                call_ask = calls.loc[call_idx, 'ask']
                put_bid = puts.loc[put_idx, 'bid']
                put_ask = puts.loc[put_idx, 'ask']
                if call_bid is not None and call_ask is not None:
                    call_mid = (call_bid + call_ask) / 2.0
                else:
                    call_mid = None
                if put_bid is not None and put_ask is not None:
                    put_mid = (put_bid + put_ask) / 2.0
                else:
                    put_mid = None
                if call_mid is not None and put_mid is not None:
                    straddle = call_mid + put_mid
                open_interest_avg = (call_oi + put_oi) / 2.0
                open_interest_check = open_interest_avg
            i += 1
        if not atm_iv:
            return {"error": "ATM IV hesaplanamadı."}
        # Opsiyon tarihleri ve IV değerlerini hesapla
        dtes = []
        ivs = []
        for exp_date, iv in atm_iv.items():
            exp_date_obj = datetime.strptime(exp_date, "%Y-%m-%d").date()
            days_to_expiry = (exp_date_obj - ref_date).days
            dtes.append(days_to_expiry)
            ivs.append(iv)
        # Eğer yalnızca bir nokta varsa, ts slope'u sıfır olarak belirleyelim
        if len(dtes) < 2:
            slope = 0
            iv30 = ivs[0]
        else:
            term_spline = build_term_structure(dtes, ivs)
            first_dte = dtes[0]
            slope = (term_spline(45) - term_spline(first_dte)) / (45 - first_dte) if (45 - first_dte) != 0 else 0
            iv30 = term_spline(30)
        price_history = stock.history(period='3mo')
        if price_history.empty:
            return {"error": "Yeterli fiyat geçmişi yok."}
        realized_vol = yang_zhang(price_history)
        iv30_rv30 = iv30 / realized_vol if realized_vol != 0 else 0
        avg_volume_val = price_history['Volume'].rolling(30).mean().dropna().iloc[-1]
        expected_move = f"{round(straddle / underlying_price * 100, 2)}%" if straddle is not None else "N/A"
        
        results = {}
        passes = {}
        if thresholds["avg_volume"]["enabled"]:
            passes["avg_volume"] = avg_volume_val >= thresholds["avg_volume"]["value"]
            results["avg_volume"] = f"{avg_volume_val} ({'PASS' if passes['avg_volume'] else 'FAIL'})"
        else:
            results["avg_volume"] = "Disabled"
            
        if thresholds["iv30_rv30"]["enabled"]:
            passes["iv30_rv30"] = iv30_rv30 >= thresholds["iv30_rv30"]["value"]
            results["iv30_rv30"] = f"{round(iv30_rv30,2)} ({'PASS' if passes['iv30_rv30'] else 'FAIL'})"
        else:
            results["iv30_rv30"] = "Disabled"
            
        if thresholds["ts_slope"]["enabled"]:
            passes["ts_slope"] = slope <= thresholds["ts_slope"]["value"]
            results["ts_slope"] = f"{round(slope,6)} ({'PASS' if passes['ts_slope'] else 'FAIL'})"
        else:
            results["ts_slope"] = "Disabled"
            
        if thresholds["open_interest"]["enabled"]:
            passes["open_interest"] = open_interest_check >= thresholds["open_interest"]["value"]
            results["open_interest"] = f"{open_interest_check} ({'PASS' if passes['open_interest'] else 'FAIL'})"
        else:
            results["open_interest"] = "Disabled"
        
        if thresholds["ts_slope"]["enabled"] and not passes.get("ts_slope", True):
            overall = "Avoid"
        elif thresholds["open_interest"]["enabled"] and not passes.get("open_interest", True):
            overall = "Avoid"
        else:
            active_checks = [k for k, v in thresholds.items() if v["enabled"]]
            pass_count = sum(1 for k in active_checks if passes.get(k, True))
            if pass_count == len(active_checks):
                overall = "Recommended"
            elif pass_count >= len(active_checks) - 1:
                overall = "Consider"
            else:
                overall = "Avoid"
        
        results["expected_move"] = expected_move
        results["overall"] = overall
        return results
        
    except Exception:
        return {"error": "İşlem sırasında bir hata oluştu."}

def run_screening(thresholds, ticker_list):
    results = {}
    ref_date = datetime.today().date()
    for ticker in ticker_list:
        rec = compute_recommendation(ticker, thresholds, ref_date)
        results[ticker] = {"ref_date": ref_date, **rec}
    return results

def main_screener():
    sg.theme("LightBlue")
    default_thresholds = {
        "avg_volume": {"enabled": True, "value": 1500000},
        "iv30_rv30": {"enabled": True, "value": 1.25},
        "ts_slope": {"enabled": True, "value": -0.00406},
        "open_interest": {"enabled": True, "value": 5000}
    }
    
    threshold_layout = [
        [sg.Text("Threshold Settings", font=("Helvetica", 14))],
        [sg.Checkbox("Avg Volume >= ", default=True, key="chk_avg_volume"), sg.Input(default_text=str(default_thresholds["avg_volume"]["value"]), key="inp_avg_volume", size=(10,1))],
        [sg.Checkbox("IV30/RV30 >= ", default=True, key="chk_iv30_rv30"), sg.Input(default_text=str(default_thresholds["iv30_rv30"]["value"]), key="inp_iv30_rv30", size=(10,1))],
        [sg.Checkbox("TS Slope <= ", default=True, key="chk_ts_slope"), sg.Input(default_text=str(default_thresholds["ts_slope"]["value"]), key="inp_ts_slope", size=(10,1))],
        [sg.Checkbox("Open Interest >= ", default=True, key="chk_open_interest"), sg.Input(default_text=str(default_thresholds["open_interest"]["value"]), key="inp_open_interest", size=(10,1))]
    ]
    
    ticker_input_layout = [
        [sg.Text("Enter Tickers (comma separated):", font=("Helvetica", 14))],
        [sg.Input(key="inp_tickers", size=(40,1))]
    ]
    
    layout = [
        [sg.Column(threshold_layout)],
        [sg.Column(ticker_input_layout)],
        [sg.Button("Run Screening"), sg.Button("Exit")],
        [sg.Text("Screening Results:", font=("Helvetica", 14))],
        [sg.Multiline("", size=(70, 20), key="ml_details")]
    ]
    
    window = sg.Window("Earnings Screener", layout, finalize=True)
    screening_results = {}
    
    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, "Exit"):
            break
        if event == "Run Screening":
            tickers_raw = values.get("inp_tickers", "")
            ticker_list = [t.strip().upper() for t in tickers_raw.split(",") if t.strip() != ""]
            if not ticker_list:
                window["ml_details"].update("Lütfen en az bir ticker giriniz.")
                continue
            user_thresholds = {
                "avg_volume": {"enabled": values["chk_avg_volume"], "value": float(values["inp_avg_volume"])},
                "iv30_rv30": {"enabled": values["chk_iv30_rv30"], "value": float(values["inp_iv30_rv30"])},
                "ts_slope": {"enabled": values["chk_ts_slope"], "value": float(values["inp_ts_slope"])},
                "open_interest": {"enabled": values["chk_open_interest"], "value": float(values["inp_open_interest"])}
            }
            def screening_worker():
                nonlocal screening_results
                screening_results = run_screening(user_thresholds, ticker_list)
            thread = threading.Thread(target=screening_worker, daemon=True)
            thread.start()
            loading_window = sg.Window("Loading", [[sg.Text("Running screening...")]], modal=True, finalize=True)
            while thread.is_alive():
                event_load, _ = loading_window.read(timeout=100)
                if event_load == sg.WINDOW_CLOSED:
                    break
            thread.join(timeout=1)
            loading_window.close()
            
            details = ""
            for ticker, res in screening_results.items():
                details += f"Ticker: {ticker}\n"
                details += f"Reference Date: {res.get('ref_date')}\n"
                if "error" in res:
                    details += f"Error: {res['error']}\n\n"
                else:
                    details += f"Avg Volume: {res.get('avg_volume')}\n"
                    details += f"IV30/RV30: {res.get('iv30_rv30')}\n"
                    details += f"TS Slope: {res.get('ts_slope')}\n"
                    details += f"Open Interest: {res.get('open_interest')}\n"
                    details += f"Expected Move: {res.get('expected_move')}\n"
                    details += f"Overall: {res.get('overall')}\n\n"
            window["ml_details"].update(details)
    
    window.close()

if __name__ == "__main__":
    main_screener()
